In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from orbit.models import LGT, DLT
from orbit.utils.dataset import load_iclaims

from orbit.utils.params_tuning import grid_search_orbit
from orbit.diagnostics.metrics import smape, mape, wmape

## Load Data

In [2]:
raw_data = load_iclaims()
data = raw_data.copy()

print(data.shape)
data.head(5)

(443, 7)


,week,claims,trend.unemploy,trend.filling,trend.job,sp500,vix
0,2010-01-03,13.386595,0.219882,-0.318452,0.117500,-0.417633,0.122654
1,2010-01-10,13.624218,0.219882,-0.194838,0.168794,-0.425480,0.110445
2,2010-01-17,13.398741,0.236143,-0.292477,0.117500,-0.465229,0.532339
3,2010-01-24,13.137549,0.203353,-0.194838,0.106918,-0.481751,0.428645
4,2010-01-31,13.196760,0.134360,-0.242466,0.074483,-0.488929,0.487404


## LGT Tuning Exampe

In [3]:
lgt = LGT(date_col='week',
          response_col='claims',
          seasonality=52,
          estimator='stan-map')

In [4]:
param_grid = {'level_sm_input': [0.3, 0.5, 0.8],
              'seasonality_sm_input': [0.3, 0.5, 0.8]}

In [5]:
best_params, tuned_df = grid_search_orbit(param_grid,
                                          model=lgt,
                                          df=data,
                                          min_train_len=100,
                                          incremental_len=100,
                                          forecast_len=20,
                                          metrics=None,
                                          criteria=None,
                                          verbose=True)

  0%|          | 0/9 [00:00<?, ?it/s]

tuning hyper-params {'level_sm_input': 0.3, 'seasonality_sm_input': 0.3}


 11%|█         | 1/9 [00:00<00:04,  1.67it/s]

tuning metric:0.0066448
tuning hyper-params {'level_sm_input': 0.3, 'seasonality_sm_input': 0.5}


 22%|██▏       | 2/9 [00:00<00:03,  2.18it/s]

...

tuning metric:0.010161
tuning hyper-params {'level_sm_input': 0.8, 'seasonality_sm_input': 0.5}


 89%|████████▉ | 8/9 [00:03<00:00,  2.28it/s]

tuning metric:0.008788
tuning hyper-params {'level_sm_input': 0.8, 'seasonality_sm_input': 0.8}


100%|██████████| 9/9 [00:03<00:00,  2.29it/s]

tuning metric:0.0079966


In [6]:
tuned_df.head()

,level_sm_input,seasonality_sm_input,metrics
0,0.3,0.3,0.006645
1,0.3,0.5,0.006033
2,0.3,0.8,0.005651
3,0.5,0.3,0.007503
4,0.5,0.5,0.006921


In [7]:
best_params

[{'level_sm_input': 0.3, 'seasonality_sm_input': 0.8}]

## DLT Tuning Example

In [8]:
dlt = DLT(date_col='week',
          response_col='claims',
          regressor_col=['trend.unemploy', 'trend.filling', 'trend.job'],
          seasonality=52,
          estimator='stan-map')

In [9]:
param_grid = {'damped_factor': [0.3, 0.5, 0.8],
              'slope_sm_input': [0.3, 0.5, 0.8]}

In [10]:
best_params, tuned_df = grid_search_orbit(param_grid,
                                          model=dlt,
                                          df=data,
                                          min_train_len=100,
                                          incremental_len=100,
                                          forecast_len=20,
                                          metrics=None,
                                          criteria=None,
                                          verbose=True)

  0%|          | 0/9 [00:00<?, ?it/s]

tuning hyper-params {'damped_factor': 0.3, 'slope_sm_input': 0.3}


 11%|█         | 1/9 [00:00<00:05,  1.56it/s]

tuning metric:0.005798
tuning hyper-params {'damped_factor': 0.3, 'slope_sm_input': 0.5}


 22%|██▏       | 2/9 [00:01<00:03,  1.90it/s]

...

tuning metric:0.0057215
tuning hyper-params {'damped_factor': 0.8, 'slope_sm_input': 0.5}


 89%|████████▉ | 8/9 [00:04<00:00,  1.88it/s]

tuning metric:0.0057388
tuning hyper-params {'damped_factor': 0.8, 'slope_sm_input': 0.8}


100%|██████████| 9/9 [00:04<00:00,  1.84it/s]

tuning metric:0.0057492


In [11]:
tuned_df.head()

,damped_factor,slope_sm_input,metrics
0,0.3,0.3,0.005798
1,0.3,0.5,0.005789
2,0.3,0.8,0.005771
3,0.5,0.3,0.005631
4,0.5,0.5,0.005726


In [12]:
best_params

[{'damped_factor': 0.5, 'slope_sm_input': 0.8}]